# Storing CSV file in S3

- Below codes are already implemented and hence you wont find the csv file or the api keys.
- CSV file: stored in cloud. Only the ones used from EDA and preprocessing will be stored locally. 
- API keys: stored securely for the app.

In [6]:
# Bucket Connection
import boto
import boto.s3.connection
from io import StringIO
import boto3
import pandas as pd

In [6]:
# The file to write in S3.

df = pd.read_csv(r'data_final.csv')
df

,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week,predicted_labels,mean_trafficVolume,mean_accidentVolume,mean_crashRate,risk_level
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday,0,204447.984615,268.707692,6.745231,low
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday,12,56487.460674,369.303371,13.129551,medium
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday,0,204447.984615,268.707692,6.745231,low
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday,0,204447.984615,268.707692,6.745231,low
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday,12,56487.460674,369.303371,13.129551,medium
...,...,...,...,...,...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday,16,120269.428571,336.224490,78.365510,high
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday,16,120269.428571,336.224490,78.365510,high
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday,16,120269.428571,336.224490,78.365510,high
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday,16,120269.428571,336.224490,78.365510,high


In [7]:
df_2 = pd.read_csv(r'kmeans_k=25.csv')
df_2

,street,traffic volume,accident count,crash_rate,latitude,longitude,risk level,Frequent_Day,Frequent_Hour,label
0,"102ND ST, Los Angeles",2238,184,8.22,33.944103,-118.254688,low,Friday,14,18
1,"103RD ST, Los Angeles",138801,531,0.38,33.942642,-118.243168,low,Tuesday,14,18
2,"104TH ST, Los Angeles",17741,239,1.35,33.941790,-118.369982,low,Saturday,17,5
3,"105TH ST, Los Angeles",14131,178,1.26,34.750118,-118.316244,low,Tuesday,1,13
4,"10TH AV, Los Angeles",3642,145,3.98,34.037828,-118.327656,low,Saturday,15,8
...,...,...,...,...,...,...,...,...,...,...
1019,"YOLANDA AV, Los Angeles",24334,76,0.31,34.195669,-118.540414,low,Monday,7,16
1020,"YORK BL, Los Angeles",271200,158,0.06,33.957271,-118.050991,low,Monday,18,14
1021,"YUCCA ST, Los Angeles",22468,204,0.91,34.103720,-118.324950,low,Monday,6,20
1022,"ZELZAH AV, Los Angeles",96051,110,0.11,34.243016,-118.523271,low,Monday,8,1


In [8]:
### Retrieving the keys - Done already

keys = []

with open('api_keys.txt', 'r') as f:
    data = f.readlines()

    for i in data:
        keys.append(str(i).replace('\n', ''))

In [9]:
# Listing all buckets

conn = boto.connect_s3(
    aws_access_key_id = keys[0],
    aws_secret_access_key = keys[1]
)

for bucket in conn.get_all_buckets():
    print("{name}\t{created}".format(
        name = bucket.name,
        created = bucket.creation_date,
    ))


# Writing file into amazon S3
s3 = boto3.client('s3', aws_access_key_id=keys[0], aws_secret_access_key=keys[1])
csv_buf = StringIO()
df.to_csv(csv_buf, header=True, index=False)
csv_buf.seek(0)
s3.put_object(Bucket='sofians3', Body=csv_buf.getvalue(), Key='data_final.csv')

sofians3	2022-04-20T12:32:08.000Z


{'ResponseMetadata': {'RequestId': '7WZ1G9TX3PRV2WGC',
  'HostId': 'xY7kRWVNywz0IdBtlwS6IM0LUrQWCebgy00ZVQzWnSxFkqChK9lxG16/0ioJBaD5oMA3fR/ukVY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xY7kRWVNywz0IdBtlwS6IM0LUrQWCebgy00ZVQzWnSxFkqChK9lxG16/0ioJBaD5oMA3fR/ukVY=',
   'x-amz-request-id': '7WZ1G9TX3PRV2WGC',
   'date': 'Mon, 25 Apr 2022 22:31:31 GMT',
   'etag': '"ca9c466ff1f0a73648060af507beec08"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"ca9c466ff1f0a73648060af507beec08"'}

In [ ]:
df_2.to_csv(csv_buf, header=True, index=False)
csv_buf.seek(0)
s3.put_object(Bucket='sofians3', Body=csv_buf.getvalue(), Key='kmeans_k=25.csv')

In [3]:
client = boto3.client('s3', aws_access_key_id=keys[0], aws_secret_access_key=keys[1])
bucket_name = 'sofians3'
object_key = 'data_final.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))
df

,DR Number,accident_address,hour,accident_latitude,accident_longitude,day_week,predicted_labels,mean_trafficVolume,mean_accidentVolume,mean_crashRate,risk_level
0,100100007,"PICO, Los Angeles",22,34.0395,-118.2656,Monday,0,204447.984615,268.707692,6.745231,low
1,100100767,"1300 N BROADWAY, Los Angeles",4,34.0695,-118.2324,Wednesday,12,56487.460674,369.303371,13.129551,medium
2,100100831,"PICO, Los Angeles",1,34.0424,-118.2718,Sunday,0,204447.984615,268.707692,6.745231,low
3,100101004,"FIGUEROA, Los Angeles",20,34.0344,-118.2714,Saturday,0,204447.984615,268.707692,6.745231,low
4,100101322,"COLLEGE, Los Angeles",16,34.0637,-118.2361,Saturday,12,56487.460674,369.303371,13.129551,medium
...,...,...,...,...,...,...,...,...,...,...,...
570367,222104843,"WINNETKA AV, Los Angeles",19,34.1857,-118.5710,Wednesday,16,120269.428571,336.224490,78.365510,high
570368,222104874,"TOPANGA CANYON BL, Los Angeles",15,34.1938,-118.6059,Monday,16,120269.428571,336.224490,78.365510,high
570369,222104886,"DE SOTO AV, Los Angeles",17,34.1740,-118.5888,Monday,16,120269.428571,336.224490,78.365510,high
570370,222104962,"AVENUE SAN LUIS, Los Angeles",19,34.1624,-118.6285,Friday,16,120269.428571,336.224490,78.365510,high


In [10]:
bucket_name = 'sofians3'
object_key = 'kmeans_k=25.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))
df

,street,traffic volume,accident count,crash_rate,latitude,longitude,risk level,Frequent_Day,Frequent_Hour,label
0,"102ND ST, Los Angeles",2238,184,8.22,33.944103,-118.254688,low,Friday,14,18
1,"103RD ST, Los Angeles",138801,531,0.38,33.942642,-118.243168,low,Tuesday,14,18
2,"104TH ST, Los Angeles",17741,239,1.35,33.941790,-118.369982,low,Saturday,17,5
3,"105TH ST, Los Angeles",14131,178,1.26,34.750118,-118.316244,low,Tuesday,1,13
4,"10TH AV, Los Angeles",3642,145,3.98,34.037828,-118.327656,low,Saturday,15,8
...,...,...,...,...,...,...,...,...,...,...
1019,"YOLANDA AV, Los Angeles",24334,76,0.31,34.195669,-118.540414,low,Monday,7,16
1020,"YORK BL, Los Angeles",271200,158,0.06,33.957271,-118.050991,low,Monday,18,14
1021,"YUCCA ST, Los Angeles",22468,204,0.91,34.103720,-118.324950,low,Monday,6,20
1022,"ZELZAH AV, Los Angeles",96051,110,0.11,34.243016,-118.523271,low,Monday,8,1
